# Employment-Unemployment Model

<br>
<br>

**Goal**:

Estimate a Markov chain using employment data

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

## Steps to "understanding the world around us" (a short digression)


Our friend Jim Savage has developed what he calls a "modern statistical workflow" for understanding the world around us. We summarize (a subset) of his steps below:

> 1. Prepare and visualize your data.
> 2. Create a generative model for the data...
>   - The first model created should be as simple as possible.
> 3. Simulate some “fake data” from your model given some parameters that you "pick out of a hat"
> 4. Fit your model to the fake data and check for the quality of the fit
> 5. Check that you were able to capture these “known unknowns”.
>   - Possibly repeat 3-5 with different methods and parameters, to get an understanding of how well the fitting procedure works
> 6. Fit the model to your real data, check the fit
> 7. Argue about the results with your friends and colleagues
> 8. Go back to 2. with a slightly richer model. Repeat.
> 9. Think carefully about what decisions will be made from the analysis, encode a loss function, and perform statistical decision analysis.

Later this semester, we will talk extensively about what it means to "fit" your model (and all of the work that it entails), but, for now, we find it sufficient to say that it's a process to help your model line up with the data that you're working to understand.

We'll do a version of steps 1-6 to help us improve our understanding of the labor data that we previously saw.

### Our plan

1. Skip -- We've previously prepared and visualized our data.
2. Develop a generative model of employment and unemployment
3. Simulate data from our generative model for given parameters
4. Fit our model to the simulated data
5. Explore different ways that we might have chosen to fit the data
6. Fit the model with the BLS data
7. Examine what our model implies for the effects of COVID on employment/unemployment

## Step 2: Develop a generative model

**A simple model of employment**

In the vein of, "the first model created should be as simple as possible", we revisit the unemployment model that we have previously discussed.

Consider a single individual that transitions between employment and unemployment

* When employed, they lose their job with probability $\beta$
* When unemployed, they find a new job with probability $\alpha$

<br>
<br>

![ModelFlowchart](model_diagram.png)


## Step 3: Simulate data from our generative model

<br>
<br>

We will break simulating data from this model into two steps:

1. Given today's state and the transition probabilities, draw from tomorrow's state
2. Given an initial state and transition probabilities, simulate an entire history of employment/unemployment


**Simulate the one-step employment transition**


In [2]:
def next_state(s_t, alpha, beta):
    """
    Transitions from employment/unemployment in period t to
    employment/unemployment in period t+1
    
    Parameters
    ----------
    s_t : int
        The individual's current state... s_t = 0 maps to
        unemployed and s_t = 1 maps to employed
    alpha : float
        The probability that an individual goes from
        unemployed to employed
    beta : float
        The probability that an individual goes from
        employed to unemployed
    """
    # Draw a random number
    u_t = np.random.rand()

    # Let 0 be unemployed... If unemployed and draws
    # a value less than lambda then becomes employed
    if (s_t == 0) and (u_t < alpha):
        s_tp1 = 1
    # Let 1 be employed... If employed and draws a
    # value less than beta then becomes unemployed
    elif (s_t == 1) and (u_t < beta):
        s_tp1 = 0
    # Otherwise, he keeps the same state as he had
    # at period t
    else:
        s_tp1 = s_t

    return s_tp1


Notice how this function depends on the Markov property!

The Markov property says $\text{Probability}(s_{t+1} | s_{t}) = \text{Probability}(s_{t+1} | s_{t}, s_{t-1}, \dots, s_0)$.

This means that, other than the transition probabilities, we only need to give the function the previous state and not the entire history.

**Testing our function**

It's always a good idea to write some simple test cases for functions that you create.

In [14]:
# Should always transition to employment from unemployment
# when alpha is 1
next_state(0, 1.0, 0.5) == 1

True

In [30]:
# Should always transition to unemployment from employment
# when beta is 1
next_state(1, 0.5, 1.0) == 0

True

**Simulate entire history**

**Note**: We will eventually allow $\alpha$ and $\beta$ to change over time, so, while we want you think of them as constant for now, we will be writing the code in a way that allows for them to fluctate period-by-period

In [31]:
def simulate_employment_history(alpha, beta, s_0):
    """
    Simulates the history of employment/unemployment. It
    will simulate as many periods as elements in `alpha`
    and `beta`
    
    Parameters
    ----------
    alpha : np.array(float, ndim=1)
        The probability that an individual goes from
        unemployed to employed
    beta : np.array(float, ndim=1)
        The probability that an individual goes from
        employed to unemployed
    s_0 : int
        The initial state of unemployment/employment, which
        should take value of 0 (unemployed) or 1 (employed)
    """
    # Create array to hold the values of our simulation
    assert(len(alpha) == len(beta))
    T = len(alpha)
    s_hist = np.zeros(T+1, dtype=int)

    s_hist[0] = s_0
    for t in range(T):
        # Step one period into the future
        s_0 = next_state(s_0, alpha[t], beta[t])  # Notice alpha[t] and beta[t]
        s_hist[t+1] = s_0

    return s_hist

**Check output of the function**

In [41]:
alpha = np.ones(50)*0.25
beta = np.ones(50)*0.025

simulate_employment_history(alpha, beta, 1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])

## Step 4: Fit your model to the fake data

There are lots of procedures that one could use to map the data into the implied parameters of your model, but we're going to proceed by simply counting the "frequencies of transitions"

Let's think about the general case. Consider an $N$-state Markov chain. The parameters of the Markov chain are the elements of the transition matrix, $P$.

$$P \equiv \begin{bmatrix} p_{11} & p_{12} & \dots & p_{1N} \\ p_{21} & \vdots & \ddots & \vdots \\ p_{N1} & p_{N2} & \dots & p_{NN} \end{bmatrix}$$

Let $\{y_0, y_1, \dots, y_T\}$ be a sequence of observations generated from the $N$-state Markov chain, then our "fitting" procedure would assign the following value to $p_{ij}$:

$$p_{ij} = \frac{\sum_{t=0}^T \mathbb{1}_{y_{t} == i} \mathbb{1}_{y_{t+1} == j}}{\sum_{t=0}^T \mathbb{1}_{y_{t} == i}}$$

**Note**: If you'd like to understand why this procedure makes sense, we recommend computing $\sum_{j=1}^N p_{ij}$ for a given $i$

**Counting frequencies**



In [55]:
def count_frequencies_individual(history):
    """
    Computes the transition probabilities for a two-state
    Markov chain

    Parameters
    ----------
    history : np.array(int, ndim=1)
        An array with the state values of a two-state Markov chain
    
    Returns
    -------
    alpha : float
        The probability of transitioning from state 0 to 1
    beta : float
        The probability of transitioning from state 1 to 0
    """
    # Get length of the simulation and an index tracker
    T = len(history)
    idx = np.arange(T)

    # Determine when the chain had values 0 and 1 -- Notice
    # that we can't use the last value because we don't see
    # where it transitions to
    zero_idxs = idx[(history == 0) & (idx < T-1)]
    one_idxs = idx[(history == 1) & (idx < T-1)]

    # Check what percent of the t+1 values were 0/1
    alpha = np.sum(history[zero_idxs+1]) / len(zero_idxs)
    beta = np.sum(1 - history[one_idxs+1]) / len(one_idxs)

    return alpha, beta

**Checking the fit**

In [56]:
def check_accuracy(T, alpha=0.25, beta=0.025):
    """
    Checks the accuracy of our fit by printing the true values
    and the fitted values for a given T
    
    Parameters
    ----------
    T : int
        The length of our simulation
    alpha : float
        The probability that an individual goes from
        unemployed to employed
    beta : float
        The probability that an individual goes from
        employed to unemployed
    """
    idx = np.arange(T)
    alpha_np = np.ones(T)*alpha
    beta_np = np.ones(T)*beta

    # Simulate a sample history
    emp_history = simulate_employment_history(alpha_np, beta_np, 0)

    # Check the fit
    alpha_hat, beta_hat = count_frequencies_individual(emp_history)
    
    print(f"True alpha was {alpha} and fitted value was {alpha_hat}")
    print(f"True beta was {beta} and fitted value was {beta_hat}")
    
    return alpha, alpha_hat, beta, beta_hat

In [70]:
check_accuracy(10_000, 0.25, 0.025)

True alpha was 0.25 and fitted value was 0.25452196382428943
True beta was 0.025 and fitted value was 0.021244309559939303


(0.25, 0.25452196382428943, 0.025, 0.021244309559939303)

Well... If we observe 10,000 months of employment history for someone then we know that we can back out the parameters of our models... Unfortunately, this is unlikely to be what our data contains...

What about for an entire lifetime of employment transitions?

In [82]:
check_accuracy(45*12, 0.25, 0.025)

True alpha was 0.25 and fitted value was 0.2631578947368421
True beta was 0.025 and fitted value was 0.028985507246376812


(0.25, 0.2631578947368421, 0.025, 0.028985507246376812)

What about for just two years?

In [110]:
check_accuracy(2*12, 0.25, 0.025)

True alpha was 0.25 and fitted value was 0.3333333333333333
True beta was 0.025 and fitted value was 0.0


(0.25, 0.3333333333333333, 0.025, 0.0)

## Step 3 and 4 (second try)

Generating data for a single individual will not give us a good chance of fitting our model accurately...

The BLS isn't basing their EU/UE rates off of a single individual, rather, they're using an entire cross-section of individuals!

Can we use a cross-section rather than a single history?

Yes, but, in order for a version of our "frequency counting" procedure to work, we need independence across individuals, i.e.

$$\text{Probability}(s_{i, t+1}, s_{j, t+1} | s_{i, t}, s_{j, t}, \alpha, \beta) = \text{Probability}(s_{i, t+1} | s_{i, t}, \alpha, \beta) \text{Probability}(s_{j, t+1} | s_{j, t}, \alpha, \beta)$$

In the previous case when we only had a single individual, the Markov property did a lot of the work to get independence for us.

When might this not be the case?

* Change in government policy results in a "jobs guarantee"
* Technological change results in the destruction of an entire industries jobs
* Recession causes increased firing across entire country

(Spoiler alert: Some of these might present problems for us... which is why we'll allow for $\alpha$ and $\beta$ to move each period)

**Simulating a cross-section**

In [ ]:
def simulate_employment_cross_section(alpha, beta, s_0, N=500):
    """
    Simulates a cross-section of employment/unemployment using
    the model we've described above.
    
    Parameters
    ----------
    alpha : np.array(float, ndim=1)
        The probability that an individual goes from
        unemployed to employed
    beta : np.array(float, ndim=1)
        The probability that an individual goes from
        employed to unemployed
    s_0 : np.array(int, ndim=1)
        The fraction of the population that begins in each
        employment state
    N : int
        The number of individuals in our cross-section
    
    Returns
    -------
    s_hist_cs : np.array(int, ndim=2)
        An `N x T` matrix that contains an individual
        history of employment along each row
    """
    # Make sure transitions are same size and get the length
    # of the simulation from the length of the transition
    # probabilities
    assert(len(alpha) == len(beta))
    T = len(alpha)

    # Check the fractions add to one and figure out how many
    # zeros we should have
    assert(np.abs(np.sum(s_0) - 1.0) < 1e-8)
    Nz = np.floor(s_0[0]*N).astype(int)

    # Allocate space to store the simulations
    s_hist_cs = np.zeros((N, T+1), dtype=int)
    s_hist_cs[Nz:, 0] = 1
    
    for i in range(N):
        s_hist_cs[i, :] = simulate_employment_history(
            alpha, beta, s_hist_cs[i, 0]
        )
    
    return s_hist_cs


In [ ]:
alpha = np.ones(1)*0.25
beta = np.ones(1)*0.025

simulate_employment_cross_section(alpha, beta, np.array([0.25, 0.75]), 10)

**Store the simulation in pandas**

Our data will typically be stored in a DataFrame, so let's keep our simulated data in a DataFrame as well

In [ ]:
def pandas_employment_cross_section(eu_ue_df, s_0, N=500):
    """
    Simulate a cross-section of employment experiences
    
    Parameters
    ----------
    eu_ue_df : pd.DataFrame
        A DataFrame with columns `dt`, `alpha`, and `beta`
        that have the monthly eu/ue transition rates
    s_0 : np.array(float, ndim=1)
        The fraction of the population that begins in each
        employment state

    Returns
    -------
    df : pd.DataFrame
        A DataFrame with the dates and an employment outcome
        associated with each date of `eu_ue_df`
    """
    # Make sure that `ue_ue_df` is sorted by date
    eu_ue_df = eu_ue_df.sort_values("dt")
    alpha = eu_ue_df["alpha"].to_numpy()
    beta = eu_ue_df["beta"].to_numpy()

    # Simulate cross-section
    employment_history = simulate_employment_cross_section(
        alpha, beta, s_0, N
    )

    df = pd.DataFrame(employment_history[:, :-1].T)
    df = pd.concat([eu_ue_df, df], axis=1)
    df = pd.melt(
        df, id_vars=["dt", "alpha", "beta"],
        var_name="pid", value_name="employment"
    )

    return df

In [ ]:
T = 24
eu_ue_df = pd.DataFrame(
    {
        "dt": pd.date_range("2019-01-01", periods=T, freq="MS"), 
        "alpha": np.ones(T)*0.25,
        "beta": np.ones(T)*0.025
    }
)

df = pandas_employment_cross_section(eu_ue_df, np.array([0.25, 0.75]), N=500)
df.head()


**Simulating the CPS**

Rather than just simulate a full history for each person, we will simulate as if they were responses to the CPS.

**Fitting to a cross-section**

We are going to continue using the "frequency of transition" idea that we previously proposed, but, we must account for the shape of the data we receive now.

Let's think about the general case. Consider an $N$-state Markov chain. The parameters of the Markov chain are the elements of the transition matrix, $P$.

$$P \equiv \begin{bmatrix} p_{11} & p_{12} & \dots & p_{1N} \\ p_{21} & \vdots & \ddots & \vdots \\ p_{N1} & p_{N2} & \dots & p_{NN} \end{bmatrix}$$

Let $\{ \{y_{i, 0}, y_{i, 1}, \dots, y_{i, T_i}\} \; \forall i \in \{0, 1, \dots, I\}\}$ be a $I$ sequences of observations generated from the $N$-state Markov chain, then our new "fitting" procedure would assign the following value to $p_{ij}$:

$$p_{ij} = \frac{\sum_{m=0}^I \sum_{t=0}^T \mathbb{1}_{y_{m, t} == i} \mathbb{1}_{y_{m, t+1} == j}}{\sum_{m=0}^I \sum_{t=0}^T \mathbb{1}_{y_{m, t} == i}}$$


In [ ]:
def count_frequencies_cross_section(history):
    """
    Computes the transition probabilities for a two-state
    Markov chain

    Parameters
    ----------
    history : np.array(int, ndim=1)
        An array with the state values of a two-state Markov chain
    
    Returns
    -------
    alpha : float
        The probability of transitioning from state 0 to 1
    beta : float
        The probability of transitioning from state 1 to 0
    """
    # Get length of the simulation and an index tracker
    T = len(history)
    idx = np.arange(T)

    # Determine when the chain had values 0 and 1 -- Notice
    # that we can't use the last value because we don't see
    # where it transitions to
    zero_idxs = idx[(history == 0) & (idx < T-1)]
    one_idxs = idx[(history == 1) & (idx < T-1)]

    # Check what percent of the t+1 values were 0/1
    alpha = np.sum(history[zero_idxs+1]) / len(zero_idxs)
    beta = np.sum(1 - history[one_idxs+1]) / len(one_idxs)

    return alpha, beta

In [ ]:
def check_accuracy(T, alpha=0.25, beta=0.025):
    """
    Checks the accuracy of our fit by printing the true values
    and the fitted values for a given T
    
    Parameters
    ----------
    T : int
        The length of our simulation
    alpha : float
        The probability that an individual goes from
        unemployed to employed
    beta : float
        The probability that an individual goes from
        employed to unemployed
    """
    idx = np.arange(T)
    alpha_np = np.ones(T)*alpha
    beta_np = np.ones(T)*beta

    # Simulate a sample history
    emp_history = simulate_employment_history(alpha_np, beta_np, 0)

    # Check the fit
    alpha_hat, beta_hat = count_frequencies_individual(emp_history)
    
    print(f"True alpha was {alpha} and fitted value was {alpha_hat}")
    print(f"True beta was {beta} and fitted value was {beta_hat}")
    
    return alpha, alpha_hat, beta, beta_hat

**Simulating the CPS**

Recall that the CPS is generated by interviewing an individual for four consecutive months, having no interviews for 8 months, and then interviewing for four additional months.

We will write code that allows us to simulate fake panels of CPS data which we will use to do similar exercises 

In [ ]:
def single_individual(eu_ue_df, pid, s_0):
    """
    Simulate a single individual's experience while in
    a fake CPS sample
    
    Parameters
    ----------
    eu_ue_df : pd.DataFrame
        A DataFrame with columns `dt`, `alpha`, and `lambda`
        that have the monthly eu/ue transition rates
    s_0 : int
        The initial state of unemployment/employment, which
        should take value of 0 (unemployed) or 1 (employed)

    Returns
    -------
    df : pd.DataFrame
        A DataFrame with the dates and an employment outcome
        associated with each date of `eu_ue_df`
    """
    # Make sure that `ue_ue_df` is sorted by date
    eu_ue_df = eu_ue_df.sort_values("dt")
    _alpha = eu_ue_df["alpha"].to_numpy()
    _lambda = eu_ue_df["lambda"].to_numpy()

    # Simulate an individual
    employment_history = simulate_employment_history(_alpha, _lambda, s_0)

    # Combine into a DataFrame
    df = pd.DataFrame(
        {
            "pid": pid,
            "dt": eu_ue_df["dt"],
            "employment": employment_history
        }
    )

    return df

In [ ]:
eu_ue_df = pd.DataFrame(
    {
        "dt": pd.date_range("2020-01-01", periods=24, freq="MS"),
        "alpha": 0.025,
        "lambda": 0.25,
    }
)

single_individual(eu_ue_df, 0, 0)

In [ ]:
def cps_single_individual(df, start_year, start_month):
    """
    Takes a simulated individual and "interviews" them
    according to the CPS schedule
    
    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame with the columns `pid`, `dt`, and
        `employment`
    start_year : int
        The year in which their interviewing begins
    start_month : int
        The month in which their interviewing begins

    Returns
    -------
    cps : pd.DataFrame
        A DataFrame with the same columns as `df` but only
        with observations that correspond to the CPS
        interview schedule for someone who starts
        interviewing in f`{start_year}/{start_month}`
    """
    # Get dates that are associated with being interviewed in
    # the CPS
    start_date_y1 = dt.datetime(start_year, start_month, 1)
    dates_y1 = pd.date_range(start_date_y1, periods=4, freq="MS")
    start_date_y2 = dt.datetime(start_year+1, start_month, 1)
    dates_y2 = pd.date_range(start_date_y2, periods=4, freq="MS")
    dates = dates_y1.append(dates_y2)

    # Filter data that's not in the dates
    cps = df.loc[df["dt"].isin(dates), :]

    return cps
    

In [ ]:
cps_single_individual(single_individual(eu_ue_df, 0, 0), 2020, 5)

**Unemployment Lecture**

* A (finite) Markov chain of unemployment
  - Describe simple model (that they briefly saw with Tom)
  - Talk about how we could use this process to simulate a version of the CPS data
  - Write code to do the simulation and store in a pandas DataFrame
* Building up to the Lake model
  - Estimate the probability associated with EU and UE
  - How do these estimates change as we increase the number of individuals in our economy?
  - Present the Lake model as what happens when we make each individual "infinitely small"
* Economic experiments
  - Go back to the graphs we made using the BLS data
  - Extract the transition rates from the data
  - Perform experiments... Answer questions like, if the rates went back to pre-COVID levels, how long until we reached pre-COVID levels of employment/unemployment?

**Want**: Given current trends in the US labor market, how long until the economy has "recovered"? We will measure this as the economy reaching the pre-COVID levels of employment/unemployment.

**Steps (from last to first)**

- Use EU/UE transition rates to forecast employment/unemployment into the future to find when employment/unemployment reach pre-COVID levels
- Forecast EU/UE transition rates into the future
- Develop a simulatable model of employment/unemployment that output employment/unemployment
- Use BLS data to determine pre-COVID levels of employment/unemployment and the history of EU/UE transition rates

![employed_unemployed](employed_unemployed.png)

## BLS Data

We have already seen how to download and manipulate the BLS data on employment/unemployment, so we take it as given in this lecture.

In [ ]:
bls = pd.read_csv("bls_clean.csv", parse_dates=["dt"])

pre_covid_employment = (
    bls.query("variable == 'employed'")
       .loc[bls["dt"].dt.year == 2019, "value"]
       .mean()
)
pre_covid_unemployment = (
    bls.query("variable == 'unemployed'")
       .loc[bls["dt"].dt.year == 2019, "value"]
       .mean()
)


In [ ]:
eu_ue_df = (
    bls.pivot(index="dt", columns="variable", values="value")
       .assign(eu=lambda x: x["eu"]/x["employed"], ue=lambda x: x["ue"]/x["unemployed"])
       .loc["2018-01-01":, ["eu", "ue"]]
       .rename(columns={"eu": "alpha", "ue": "lambda"})
       .reset_index()
)

eu_ue_df.head()
